# Thermal coupling on coincident domains

Heat exchange between two domains initially at different temperatures with :
- same domain `dom` used for both problems, with `PolyMAC_P0P1NC` discretization for both problems (no interpolator)
- different domains `dom1` and `dom2` used for each problem, with `PolyMAC_P0P1NC` discretization for both problems
    - using `metis` independently on each domain, and using `OverlapDEC`
    - using the `med` partitionner (coincident partition for both domains)
        - with `OverlapDEC`
        - with local `remapper`
- different domains `dom1` and `dom2` used for each problem, with `PolyMAC_P0P1NC` for `pb1`, and `VEF` for `pb2`
    - using `metis` independently on each domain, and using `OverlapDEC`
    - using the `med` partitionner (coincident partition for both domains)
        - with `OverlapDEC`
        - with local `remapper`

In [ ]:
from trustutils.jupyter import run

run.introduction("Yannick Gorsse")
run.TRUST_parameters()

In [ ]:
run.reset()
run.initBuildDirectory()

part_med = "partitionneur fichier_med { file dom1_0000.med }"
part_met = "partitionneur metis { nb_parts 2 }"
np = 2
nx = 50

# single domain case
tc = run.addCaseFromTemplate("jdd_single_dom.data", "single_domain", {"N" : nx}, nbProcs=np)
tc.partition()

# dual domain cases
for meshname, mesht in (("same", ""), ("different", "trianguler_h dom2")):
    for pn, part in (("metis", part_met), ("med", part_med)):
        for dec in [0, 1]:
            if part == part_met and not dec: continue
            tc = run.addCaseFromTemplate("jdd_two_dom.data", f"{meshname}meshes/part{pn}/dec{dec}", {"N" : nx, "option_maillage_t" : mesht, "part" : part, "dec" : "" if dec else "without_dec"}, nbProcs=np)
            tc.partition()
# dual domain cases (VEF)
for meshname, mesh1, mesh2 in (("same", "trianguler_h dom1", ""), ("different", "", "")): # raffiner_simplexes dom2
    for pn, part in (("metis", part_met), ("med", part_med)):
        for dec in [0, 1]:
            if part == part_met and not dec: continue
            tc = run.addCaseFromTemplate("jdd_poly_vef.data", f"VEF/{meshname}meshes/part{pn}/dec{dec}", {"N" : nx, "option_maillage1" : mesh1, "option_maillage2" : mesh2, "part" : part, "dec" : "" if dec else "without_dec"}, nbProcs=np)
            tc.partition()

run.runCases()
run.tablePerf()

In [ ]:
from trustutils import plot

a = plot.Graph("Residuals")
a.addResidu("single_domain/PAR_jdd_single_dom.dt_ev", label="single domain")
for meshname, _ in (("same", ""), ("different", "trianguler_h domt")):
    for pn, part in (("metis", part_met), ("med", part_med)):
        for dec in [0, 1]:
            if part == part_met and not dec: continue
            a.addResidu(f"{meshname}meshes/part{pn}/dec{dec}/PAR_jdd_two_dom.dt_ev", label=f"{meshname}meshes/part{pn}/dec{dec}")
            a.addResidu(f"VEF/{meshname}meshes/part{pn}/dec{dec}/PAR_jdd_poly_vef.dt_ev", label=f"VEF/{meshname}meshes/part{pn}/dec{dec}")

a.scale(yscale="log")

In [ ]:
a = plot.Graph(f"Single domain")
for pb in [1, 2]:
    a.addPoint(f"{run.BUILD_DIRECTORY}/single_domain/PAR_jdd_single_dom_T{pb}.son", label=f"pb {pb}")
for meshname, mesht in (("same", ""), ("different", "trianguler_h domt")):
    for pn, part in (("metis", part_met), ("med", part_med)):
        for dec in [0, 1]:
            if part == part_met and not dec: continue
            a = plot.Graph(f"{meshname}meshes/part{pn}/dec{dec}")
            for pb in [1, 2]:
                a.addPoint(f"{run.BUILD_DIRECTORY}/{meshname}meshes/part{pn}/dec{dec}/PAR_jdd_two_dom_T{pb}.son", label=f"pb {pb}")
                a.addPoint(f"{run.BUILD_DIRECTORY}/VEF/{meshname}meshes/part{pn}/dec{dec}/PAR_jdd_poly_vef_T{pb}.son", label=f"VEF - pb {pb}", marker="x")
